import library

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import h5py

import random

load data from h5
- model.h5 link = https://drive.google.com/drive/folders/1VquNOLLXzaBVts21QKkU8TFLH2sq20Xm?usp=sharing

In [2]:
with h5py.File('./'file path'/model.h5', 'r') as hdf:
    R = np.array(hdf.get('R'))
    W = np.array(hdf.get('W'))
    X = np.array(hdf.get('X'))
    index_book = np.array(hdf.get('index_book'))

initialize variable

In [13]:
rows = 2
cols = R.shape[0] # 6k row
size = rows*cols
amount = 5000

R_user = np.zeros([rows, cols])
R_target = np.zeros([rows, cols])
rate_index = np.zeros([cols, rows]) 
    #row 0 = goodread index (float, hrs jadi int)
    #row 1 = feature vector-calculated rating
similarity_batch = np.zeros(amount)
    
for i in range(cols):
    R_user[0][i], R_target[0][i] = i, i #index x-axis
    
    rate_index[i][0] = index_book[i][0]

# initialize cosine similarity function
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)

# initialize dictionary books_id & index matrix
dict_buku = {}
for key in index_book:
    dict_buku[key[0]] = key[2]

x = np.array(range(0,R.shape[1]+1))
x = x.tolist()
idx_batch = random.sample(x, amount)

functions

In [14]:
def random_batch():
    x = np.array(range(0,R.shape[1]+1))
    x = x.tolist()
    idx_batch = random.sample(x, amount)

In [15]:
# user_books = array

def predict_batch (user_books):
    
    global R_user, R_target, rate_index, similarity, idx_batch
    
    random_batch()
    
    # convert user_books to user sparse array
    for item in user_books:
        index = dict_buku[item]
        R_user[1][index] = 1 # 1 for marked books index
    
    # compare and calculate cosine similarity between user rating
    #     and all other user, then pick the least value (very 
    #     similar) with the user rating
    #          - -1 = mirip bgt
    #          - +1 = beda bgt
    for i in range(len(idx_batch)): #loop through all batched user vector
        idx = idx_batch[i]
        R_target[1] = R[:, idx]
        similarity_batch[i] = cosine_loss(R_target, R_user).numpy()    
    
    #index of the most similar vector
    idx_user = similarity_batch.argmin()
    idx_user = idx_batch[idx_user]
    
    # calculate feature vector of the very similar user with all the books.
    for i in range(R.shape[0]): # loop all books
        rate_index[i][1] = np.dot(W[idx_user], X[i])
    
    # sort the calculated rating array descendingly based on the second column (rating)
    index_kolom = 1 #second column
    rate_index = sorted(rate_index, key=lambda a_entry: a_entry[index_kolom], reverse=True) 
    rate_index = np.array(rate_index)
    
    # cast float to int, 1st column only
    sorted_rec = rate_index.astype(int)
    
    #reset R_user[1,:] to zeros
    R_user[1] = [0]*cols
    
    # reset index
    for i in range(cols):
        rate_index[i][0] = index_book[i][0]

    return sorted_rec[:,0]

execute

In [16]:
arr = [2983562, 11870085,219919, 3049, 5107]
result = predict_batch(arr)
result #recommended books id

array([22504701, 24019187, 18687079, ..., 13151637, 13151283, 18363068])

In [ ]:
books_id = request.json['book']
print(books_id)

## merging 2 array & remove duplicates

In [2]:
def merge_alternatively(lst1, lst2):
    if not lst1:
        return lst2
    if not lst2:
        return lst1
    return [lst1[0], lst2[0]] + merge_alternatively(lst1[1:], lst2[1:])

lst1 = [1, 2, 3, 4, 5] #array "collab"
lst2 = ['a', 'b', 'c', 4, 5] #array "content"

merged = merge_alternatively(lst1, lst2)

# merge duplicate while maintaining order
res = [i for n, i in enumerate(merged) if i not in merged[:n]]
print("List after removing duplicate elements: ", res)



List after removing duplicate elements:  [1, 'a', 2, 'b', 3, 'c', 4, 5]
